In [19]:
%matplotlib inline
%pylab inline


Populating the interactive namespace from numpy and matplotlib


/home/mohammad/.local/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [20]:
pylab.rcParams['figure.figsize'] = (8, 6)
matplotlib.rcParams.update({'font.size':16})

In [21]:
import sys
import os
from collections import defaultdict
import numpy as np
import pandas as pd
import json

In [22]:
def generate_dash_data_format(df, feature_list, target, regions, outfilename, region_column="app"):
    feature_list_and_target = feature_list[:]
    feature_list_and_target.append(target)
    
    with open(outfilename, 'w') as txtfile:   
        s = ','
        s += ','.join([str(item) for item in regions])
        s += '\n'
        txtfile.write(s)

        for feat in feature_list_and_target:
            if feat == region_column:
                continue
            row=feat# + ",\""
            for reg in regions:
                row += ",\""
                ## GEt all rows that have the same kernel/region name
                df_tmp = df[(df[region_column] == reg )] # DEBUG for empty values              
                vals = []
                
                ## Now, make their values into a list
                [vals.append(v) for v in df_tmp[feat].values]
                s = ','.join([str(v) for v in vals])
                row += s + "\""
            row += '\n'
            txtfile.write(row)    

In [23]:
def func(var1, var2):
    return (var1 / var2) ##This function can create an arbitrary combination of the two variables passed.

In [25]:
################### AMD Q2: AAC data on MI-50
import os.path
from os import path

def read_input_manual4(dirname, model, optimizer, filelist):
    input_prefix = [dirname+'/'] 
    
    workloads = dict()
    
    for arch in range(len(input_prefix)):
        print('ARCH: ', arch, input_prefix[arch])
        input_file_list = []
        for ind in range(len(filelist)):
            filename = input_prefix[arch] + filelist[ind]
            print('IND: ', ind, filename)
            if path.exists(filename):
                input_file_list.append(filename)
                filename=""
            else:
                print(filename, "didn't exist")
        workloads[arch] = input_file_list
    ##TODO: Change this
    output_prefix = ['/home/Mohammad/pegasus_tutorial/spider-output/dash_format_'+model+'_'+optimizer]#,
#                     '/Users/tanzima/Research/Stash/dashing-analysis-framework/data/gilgamesh/dash_format_aac_basic_lstm',
#                     '/Users/tanzima/Research/Stash/dashing-analysis-framework/data/gilgamesh/dash_format_aac_rnn']

    return workloads, output_prefix # target_list and filter_list
    

In [26]:
model_name = 'kickstart'
opt = '1000genome'
filelist=[
    # 'kickSampleNamd.xlsx'
    '0d3a50e8-d8f9-4ad1-a3d0-cceba3c9e6e1'
]
dirname = '/home/mohammad/pegasus_tutorial/KibanaDataFetch/kickstart'
# workloads, output_prefix = read_input_manual4('/Users/tanzima/Research/AMD/DATA/mi-50/DATA-aac-july21/gd/prof/', 'basic_lstm', "gd", filelist)
workloads, output_prefix = read_input_manual4(dirname, model_name, opt, filelist)


########## Adam optimizer
# filelist = ['s25-b2-l1024.csv', 's25-b2-l1536.csv', 's25-b32-l2048.csv', 's25-b32-l2560.csv',  's25-b32-l4096.csv', 's25-b64-l2048.csv']#, 'rnn/out-rnn-rnn-s30-b32-l128.csv']
# workloads, output_prefix = read_input_manual4('/Users/tanzima/Research/AMD/DATA/mi-50/DATA-aac-july21/adam/prof/', 'basic_lstm', "adam", filelist)
# workloads, output_prefix = read_input_pennant_mi100()
print(workloads)
#print(pd.read_csv(workloads[0][0]))

ARCH:  0 /home/mohammad/pegasus_tutorial/KibanaDataFetch/kickstart/
IND:  0 /home/mohammad/pegasus_tutorial/KibanaDataFetch/kickstart/0d3a50e8-d8f9-4ad1-a3d0-cceba3c9e6e1
{0: ['/home/mohammad/pegasus_tutorial/KibanaDataFetch/kickstart/0d3a50e8-d8f9-4ad1-a3d0-cceba3c9e6e1']}


In [27]:
def region_list_parser(region_list):
    for i, kernelName in enumerate(region_list):
        templateIdx = kernelName.find("<")
        if templateIdx == -1:
            continue
        namespaceIdx = kernelName.rfind("::", 0, templateIdx)
        assert(namespaceIdx != -1) # Shouldn't ever happen.
        region_list[i] = kernelName[namespaceIdx+2:templateIdx]
    print(region_list)
    return region_list

In [28]:
def changeRange(ts):
    b = ts - min(ts)
    # a= numpy.modf(b)
    # return a[1].astype(int)
    return b

In [ ]:
def normalize(feature_name):
    max_value = feature_name.max()
    min_value = feature_name.min()
    result = (feature_name - min_value) / (max_value - min_value)
    return result

In [29]:
def find_common_features_and_regions(df_dict_param, filter_list, region_column="app"):
    # df_dist: list of workloads within a model
    # filter_list: columns (metrics/counters) we don't want

    features_dict = defaultdict(int)
    common_region_dict = defaultdict(int)
    regions_list_with_dup = [] # "region" = kernel
    feature_list = [] # "feature" = column (metric/counter)
    
    #For each file
    for key, df_instance in df_dict_param.items():
        #read in the file
        df_original2 = df_instance.copy()
        ## REMOVE all columns that contain only 0s
        df_original2 = df_original2.loc[:, (df_original2 != 0).any(axis=0)]
        for v in df_original2.columns:
            features_dict[v] += 1
            
        for v in df_original2[region_column]:
            common_region_dict[v] += 1
            #regions_list_with_dup.append(v)
            
    ## Finding the unique names of regions since regions_list_with_dup may have duplicate region names.
    ##Here, region == kernel
#     region_list = list(set(regions_list_with_dup))

    ## FINDING COMMON FEATURES
    common_feature_list = []
    for k, v in features_dict.items():
        if k in filter_list:
            continue
        if v == len(df_dict_param):
            common_feature_list.append(k)

            
    ## FINDING COMMON Regions
    common_region_list = []
    for k, v in common_region_dict.items():
        if v == len(df_dict_param):
            common_region_list.append(k)

        
    return common_feature_list, common_region_list


In [32]:
## Reading and saving input
df_dict_input = {}
total_file_count = 0
filter_list = ['@version', 'wf_label','rss', 'site', 'type', 'condor_job_id','exe', 'hostname', 'task_id', 'xformation', 'wf_uuid']
for ind in range(len(workloads)):
    file_name_list = workloads[ind]
    for fname in file_name_list:
        if not path.exists(fname):
            print(fname, ' does not exist....')
            continue
        else:
            with open(fname) as f:
                temp_data = f.read()
            f.close()
            lst = json.loads(temp_data)
            df_original2 = pd.DataFrame(lst)

            df_original2['@timestamp'] = pd.to_datetime(df_original2['@timestamp'], format="%Y-%m-%dT%H:%M:%S.%f")#.sort_values()
            newdf = df_original2.sort_values(['@timestamp'], ascending = (True))
            print('----------', newdf.columns)
            print('----------')
             
            newdf['tsTrans'] = changeRange(df_original2['ts'])
            newdf['iowaitT'] = normalize(newdf['iowait'])

            newdf['vmPerSec'] = newdf['vm'] / newdf['tsTrans']
            newdf = newdf.drop(filter_list, axis=1)
            newdf = newdf.loc[:, (newdf != 0).any(axis=0)]
            
            newdf.replace([np.inf, -np.inf], np.nan, inplace=True)
            newdf = newdf.fillna(0)
            
            
            newdf = newdf.drop(['procs', '@timestamp', 'event', 'rank', 'threads', 'tsTrans'], axis=1)
            print(newdf.columns)

            df_dict_input[total_file_count] = newdf
            total_file_count += 1
print(total_file_count)
print(df_dict_input)

---------- Index(['dag_job_id', 'pid', 'vm', '@version', 'rank', 'bsend', 'site', 'syscw',
       'type', 'wf_label', 'bwrite', 'exe', 'rchar', 'threads', 'utime',
       'syscr', '@timestamp', 'xformation', 'wf_uuid', 'task_id', 'hostname',
       'brecv', 'procs', 'event', 'iowait', 'wchar', 'bread', 'ts', 'stime',
       'condor_job_id', 'rss'],
      dtype='object')
----------
Index(['dag_job_id', 'pid', 'vm', 'syscw', 'bwrite', 'rchar', 'utime', 'syscr',
       'iowait', 'wchar', 'bread', 'ts', 'stime', 'vmPerSec'],
      dtype='object')
1
{0:              dag_job_id    pid      vm  syscw  bwrite    rchar  utime  syscr  \
0        namd_ID0000002  11045  541940    115  167936   182203   5.39    236   
23       namd_ID0000002  23950  541776     22  167936   185454   5.95    262   
19       namd_ID0000002  11046  541916     22  163840   181975   6.29    233   
1        namd_ID0000002  23953  541776     22  167936   185453   5.71    264   
2        namd_ID0000002  11047  541916     22

In [25]:
import copy
# import plotly.express as px
tmpdf = copy.deepcopy(df_dict_input[0])
# # tmpdf['bwritePerT'] = tmpdf['bwrite'] / tmpdf['bwriteT']
# # tmpdf['breadPerT'] = tmpdf['bread'] / tmpdf['breadT']


# # tmpdf = tmpdf.drop(['pid', 'utime', 'syscw','event', 'stime', 'utimeT', 'stimeT', 'iowait', 'iowaitT', 'bwrite', 'bwriteT'], axis=1)


# fig = px.scatter_matrix(tmpdf)
# fig.show()

### Decompose workflow based on the characteristics of individual jobs. Each job 

In [28]:
import plotly.graph_objects as go

categories = ['bread','bwrite','vmPerSec',
              'iowaitT', 'vm']
# categories = ['bwritePerT','vmTransPerT',
#               'iowaitPerT', 'vm']


fig = go.Figure()

import pandas as pd
from sklearn import preprocessing

# print(tmpdf)
x = tmpdf.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=tmpdf.columns)
tmpdf = df
#set opacity parameter based on index
opacity_list = [0.2, 0.4, 0.6, 0.8, 1.0]
for ind in range(0,len(tmpdf['ts'])):

    rlist = []

    #Restructure data to have rlist = [i-th value from each column]
    for cols in categories:
        rlist.append(tmpdf[cols][ind])

    #Add the i-th plot to the radar chart
    fig.add_trace(go.Scatterpolar(
      r=rlist,
      theta=categories,
      fill='toself',
      opacity=opacity_list[ind%5],
      name='A'+str(ind)
    ))
#     fig = px.line_polar(tmpdf, r=categories, theta=categories, line_close=True)
    #Add interactive legends
    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, 1]
        )),
      showlegend=True
    )

#Show plot
fig.show()


### The following code block for calculating the standard deivations across indexed variables to replace channel counters.

In [25]:
## Input processing and finding common features, and create a list of data frames for each model.
# #target_list = ['score']

## Create derived target metrics
#For each file
##MOVE target_list and filter_list to be a list that is returned from read_input
target_list = ['ts', 'GPUBusy', 'VALUBusy', 'MemUnitBusy', 'VALUUtilization', 'ratio1', 'ratio2']
filter_list = ['BeginNs', 'EndNs', 'DispatchNs', 'BeginNs', 'EndNs', 'CompleteNs', 'Index','queue-id','queue-index', 'pid', 'tid', 'grd', 'wgr', 'lds', 'vgpr', 'sgpr', 'fbar', 'sig', 'obj']

###### Decide which input dictionary we want to use, the one with stdDev variables or individual channels?

# selected_df_dict = copy.deepcopy(df_dict_input)
selected_df_dict = copy.deepcopy(df_with_stdDev_vars)


common_top_region_list = []
df_dict_updated = {}
cumulative_workload_id = 0
feature_list = {} ######### CHECK
region_list = {} ################### CHECK
shorten_region_list = {} ################### CHECK
model_index = 0  # wot
print(len(workloads))
for model_index in range(len(workloads)): #For each model
    
    df_subdir = {}
    for wl in range(len(workloads[model_index])): #For each input parameter in a model
        df_subdir[wl] = selected_df_dict[cumulative_workload_id].copy()
        cumulative_workload_id += 1
    #Find the common regions and features. Should be consistent within a model.
    ## TODO: Make a dictionary of feature list per model
    feature_list[model_index], region_list[model_index] = find_common_features_and_regions(df_subdir, filter_list, "KernelName")
#     shorten_region_list[model_index] = region_list_parser(region_list[model_index].copy())
    shorten_region_list[model_index] = region_list[model_index]
    
    cumulative_df = pd.DataFrame(columns=feature_list[model_index])
    
    for wl in range(len(workloads[model_index])): #For each input parameter in a model
        #read in the file
        df_original2 = df_subdir[wl].copy() #The dataset is already there. 
        ## REMOVE all columns that contain only 0s
        df_original2 = df_original2.loc[:, (df_original2 != 0).any(axis=0)]

        #Add new targets-- for AMD work, the target 'ts' is computed based on 'BeginNs', 'EndNs'
        df_original2['ts'] = df_original2['EndNs'] - df_original2['BeginNs']
        #df_original2['ratio1'] = 100*df_original2['VALUBusy'] / df_original2['GPUBusy'] #100*df_original2['GRBM_GUI_ACTIVE']/df_original2['GRBM_COUNT']
        #df_original2['ratio2'] = 100*df_original2['VALUBusy'] / df_original2['MemUnitBusy'] #100*df_original2['GRBM_GUI_ACTIVE']/df_original2['GRBM_COUNT']
#         df_original2['shortenKernelName'] = #region_list_parser(df_original2['KernelName'].copy())#region_list[model_index].copy())
        
        ## Just keep the top 5 regions according to their 'ts'
#         df_original2.sort_values(by=['ts'], inplace=True, ascending=False)
        
        ## Only keep those rows that belong to the common region names
#         for reg in shorten_region_list[model_index]:
#             df_original2 = df_original2[(df_original2['shortenKernelName'] == reg )] 
        
        ## Now, among those common regions, choose top 15 kernels.
#         top_regions = (df_original2.head(15))['KernelName']
#         region_list = top_regions
        ## Now, create a list with the top kernel names 
        common_top_region_list.append([reg for reg in region_list[model_index]])

        
        # Now, appending all rows from different workloads to a stack workloads. Concat seems to be the way.
        cumulative_df = cumulative_df.append(df_original2, ignore_index = True)
        print(wl, df_original2['MemUnitBusy'])        

    df_dict_updated[model_index] = cumulative_df
        
        
for numx in df_dict_updated[0]['MemUnitBusy']: 
    print(len(df_dict_updated[0]['MemUnitBusy']), numx)

# print(common_top_region_list[0])
common_region_list = list(set(common_top_region_list[0]))
# print(common_region_list)

1
0 0      3.587133
1      2.820570
2     26.697662
3      6.041099
4      7.370629
5      0.000000
6      0.627186
7      0.317365
8      0.524976
9      8.000000
10     5.198301
11     0.000000
12     0.445559
Name: MemUnitBusy, dtype: float64
1 0     0.039321
1     0.085417
2     2.647113
3     1.412494
4     3.718074
5     0.150142
6     0.000000
7     0.049850
8     0.118762
9     0.064656
10    1.000000
11    2.027346
12    0.000000
13    0.052104
Name: MemUnitBusy, dtype: float64
2 0      4.186853
1      3.016892
2     16.334386
3     12.411254
4      2.968864
5      0.841799
6      1.000000
7      0.471181
8      0.184631
9      0.387865
10     3.000000
11     6.060330
12     0.000000
13     0.163903
Name: MemUnitBusy, dtype: float64
3 0     0.008631
1     0.015912
2     1.544256
3     1.000000
4     3.417208
5     0.516650
6     0.000000
7     0.057867
8     0.113772
9     0.047119
10    0.000000
11    2.085657
12    0.000000
13    0.013146
Name: MemUnitBusy, dtype: float64
4 

In [26]:
## Input processing and finding common features

#target_list = ['score']

## Create derived target metrics
#For each file
# target_list = ['ts', 'GPUBusy', 'VALUBusy', 'MemUnitBusy', 'VALUUtilization', 'ratio1', 'ratio2']
target_list = ['ts', 'GPUBusy', 'MemUnitBusy']

# config_list = ['basic_lstm', 'lstm', 'rnn']
config_list = ['pennent']
# target_list = ['ts']
cumulative_index = 0
print(len(df_dict_updated))
for model_index in range(len(df_dict_updated)):#range(len(workloads)): #For each model
    df_original2 = df_dict_updated[model_index]
    for target in target_list:
        df_original3 = df_original2[feature_list[model_index]]
        ##Make sure to fill up all Nan values with 0
        df_original3 = df_original3.fillna(0)

        #Add a target at a time to original3
        df_original3[target] = df_original2[target]

        outfilename = output_prefix[model_index] + '_' + target+'.csv'
        print('Generating ' + outfilename + '...')
        #generate_dash_data_format(df_original3, feature_list, target, region_list, outfilename, "KernelName")
        ## TODO: PASS the following function feature_list[ind] and region_list[ind]
        generate_dash_data_format(df_original3, feature_list[model_index], target, region_list[model_index], outfilename, "KernelName")
        
        #generate_config_block_for_this_input(config_list[model_index] + '_'+target, outfilename, target)
print("Done")

1
Generating /Users/tanzima/Research/Stash/dashing-analysis-framework/data/aac/dash_format_aac_basic_lstm_adam_ts.csv...
Generating /Users/tanzima/Research/Stash/dashing-analysis-framework/data/aac/dash_format_aac_basic_lstm_adam_GPUBusy.csv...
Generating /Users/tanzima/Research/Stash/dashing-analysis-framework/data/aac/dash_format_aac_basic_lstm_adam_MemUnitBusy.csv...
Done


In [27]:
########## AMD Q2 block-- generate 1 dash file per input config or Workload. This is so that we can correlate the resource
#### importance values with FOM

##MOVE target_list and filter_list to be a list that is returned from read_input
target_list = ['ts', 'GPUBusy', 'VALUBusy', 'MemUnitBusy', 'VALUUtilization', 'ratio1', 'ratio2']
filter_list = ['BeginNs', 'EndNs', 'DispatchNs', 'BeginNs', 'EndNs', 'CompleteNs', 'Index','queue-id','queue-index', 'pid', 'tid', 'grd', 'wgr', 'lds', 'vgpr', 'sgpr', 'fbar', 'sig', 'obj']

###### Decide which input dictionary we want to use, the one with stdDev variables or individual channels?

# selected_df_dict = copy.deepcopy(df_dict_input)
selected_df_dict = copy.deepcopy(df_with_stdDev_vars)
df_subdir = {}
df_dict_updated = {}
cumulative_workload_id = 0
feature_list = {} ######### CHECK
region_list = {} ################### CHECK
shorten_region_list = {} ################### CHECK


for model_index in range(len(workloads)): #For each model lstm, basic_lstm, rnn, ....
    for wl in range(len(workloads[model_index])): #For each input parameter in a model
        df_subdir[wl] = selected_df_dict[cumulative_workload_id].copy()
        feature_list[wl], region_list[wl] = find_common_features_and_regions(df_subdir, filter_list, "KernelName")
        shorten_region_list[wl] = region_list[wl]

        df_original2 = df_subdir[wl].copy() #The dataset is already there. 
        ## REMOVE all columns that contain only 0s
        df_original2 = df_original2.loc[:, (df_original2 != 0).any(axis=0)]

        #Add new targets-- for AMD work, the target 'ts' is computed based on 'BeginNs', 'EndNs'
        df_original2['ts'] = df_original2['EndNs'] - df_original2['BeginNs']
        df_original2['ratio1'] = 100*df_original2['VALUBusy'] / df_original2['GPUBusy'] #100*df_original2['GRBM_GUI_ACTIVE']/df_original2['GRBM_COUNT']
        df_original2['ratio2'] = 100*df_original2['VALUBusy'] / df_original2['MemUnitBusy'] #100*df_original2['GRBM_GUI_ACTIVE']/df_original2['GRBM_COUNT']

        #generate_dash_data_format(df_original2, feature_list[wl], "ts", region_list[wl], "sample_dash.csv", "KernelName")
        df_dict_updated[cumulative_workload_id] = df_original2#.deepcopy()

        cumulative_workload_id += 1



In [28]:
print (df_dict_updated[0]['MemUnitBusy'], df_dict_updated[0]['MemUnitBusy'])
    
            

0      3.587133
1      2.820570
2     26.697662
3      6.041099
4      7.370629
5      0.000000
6      0.627186
7      0.317365
8      0.524976
9      8.000000
10     5.198301
11     0.000000
12     0.445559
Name: MemUnitBusy, dtype: float64 0      3.587133
1      2.820570
2     26.697662
3      6.041099
4      7.370629
5      0.000000
6      0.627186
7      0.317365
8      0.524976
9      8.000000
10     5.198301
11     0.000000
12     0.445559
Name: MemUnitBusy, dtype: float64


In [29]:
print (df_dict_input[0]['MemUnitBusy'], df_dict_input[0]['MemUnitBusy'])

0      3.587133
1      2.820570
2     26.697662
3      6.041099
4      7.370629
5      0.000000
6      0.627186
7      0.317365
8      0.524976
9      8.000000
10     5.198301
11     0.000000
12     0.445559
Name: MemUnitBusy, dtype: float64 0      3.587133
1      2.820570
2     26.697662
3      6.041099
4      7.370629
5      0.000000
6      0.627186
7      0.317365
8      0.524976
9      8.000000
10     5.198301
11     0.000000
12     0.445559
Name: MemUnitBusy, dtype: float64


In [30]:
for ind in range(len(workloads)):
    print(region_list)
len(region_list[0])

{0: ['BiasGradNHWC_SharedAtomics', 'BiasNHWCKernel', 'Cijk_Ailk_Bjlk_SB_MT128x128x8_SE_1LDSB0_APM1_AF0EM1_AF1EM1_AMAS2_ASBE01_ASCE01_ASEM1_BL0_DTL0_DVO0_EPS0_FL0_GRVW4_GSU1_ISA000_IU1_K1_KLS_LBSPP0_LPA0_LPB0_LDL1_LRVW4_MDA2_NLCA1_NLCB1_ONLL1_PBD0_PK0_PGR1_PLR1_RK0_SIA1_SU32_SUM0_SUS256_SRVW0_SVW4_SNLL0_TT8_8_TLDS0_USFGRO0_VAW1_VS1_VW4_WSGRA0_WSGRB0_WG16_16_1_WGM1.kd', 'Cijk_Ailk_Bljk_SB_MT64x64x16_SE_1LDSB0_APM1_AF0EM1_AF1EM1_AMAS3_ASBE01_ASCE01_ASEM1_BL1_DTL0_DVO0_EPS1_FL1_GRVW4_GSU1_ISA908_IU1_K1_KLA_LBSPP0_LPA0_LPB4_LDL1_LRVW4_MDA2_NLCA1_NLCB1_ONLL1_PBD0_PK0_PGR1_PLR1_RK0_SIA1_SU32_SUM0_SUS256_SRVW0_SVW4_SNLL1_TT4_8_TLDS0_USFGRO0_VAW1_VS1_VW4_WSGRA0_WSGRB0_WG16_8_2_WGM8.kd', 'Cijk_Alik_Bljk_SB_MT64x64x16_SE_1LDSB0_APM1_AF0EM1_AF1EM1_AMAS3_ASBE01_ASCE01_ASEM1_BL1_DTL0_DVO0_EPS1_FL1_GRVW4_GSU1_ISA908_IU1_K1_KLA_LBSPP0_LPA4_LPB4_LDL1_LRVW4_MDA2_NLCA1_NLCB1_ONLL1_PBD0_PK0_PGR1_PLR1_RK0_SIA1_SU32_SUM0_SUS256_SRVW0_SVW4_SNLL1_TT4_4_TLDS0_USFGRO1_VAW1_VS1_VW4_WSGRA0_WSGRB0_WG16_16_1_WGM8.k

13

In [31]:
for key, val in df_dict_updated.items():
    print(len(val))

13
14
14
14
13
13
13
14
13
13


In [32]:
import pandas as pd

#create two DataFrames

df1 = pd.DataFrame({'player': ['A', 'B', 'A', 'B', 'B'],
                    'points':[12, 5, 13, 17, 27],
                   'extra':[1, 5, 3, 7, 2]})

df2 = pd.DataFrame({'player': ['F', 'G', 'H', 'I', 'J'],
                    'points':[24, 26, 27, 27, 12]})

#"stack" the two DataFrames together
df3 = pd.concat([df1,df2], axis=0, ignore_index=True)

print (df1)
# A = df1.append(df2, ignore_index=True)
# print(A.append(df3, ignore_index=True))

# df = df1.groupby(['player']).agg({'extra':'mean','points':'mean'})
df = df1.groupby(['player']).agg(func='mean')
df = df.reset_index()
print (df)

# print(df3)

  player  points  extra
0      A      12      1
1      B       5      5
2      A      13      3
3      B      17      7
4      B      27      2
  player     points     extra
0      A  12.500000  2.000000
1      B  16.333333  4.666667


In [33]:
generate_dash_data_format(df_dict_updated[0], feature_list[0], "ts", region_list[0], "sample_dash.csv", "KernelName")

In [34]:
print(len(df_dict_updated))

10


In [35]:
def generate_config_block_for_this_input(config_name, datafilename, target):
    print(config_name + ':')
    print('    data: ', datafilename)
    print('    tasks: ')
    print('      - modules.resource_score.compute_rsm_task_all_regions')
    print('      - viz.barchart.create_rsm_error_barchart')
    print('      - viz.barchart.create_rsm_percent_barchart')
    print('      - viz.sunburst2.sunburst')
    print('      - viz.linechart3.plot_raw_target_values')
    print('      - viz.linechart3.raw_values_per_config')
    print('    name: ', target)
    print('    target: ', target)
    
    if target == 'ts':
        print('    compute_target: modules.compute_target.compute_runtime')
    else:
        print('    compute_target: modules.compute_target.compute_inverse_target')
    print('##############################')
   

    

In [ ]:
## AMD Q2: OUTPUT PROESSING: 1 file per input per model.

#target_list = ['score']

## Create derived target metrics
#For each file
filelist = ['s30-b32-l128', 's30-b128-l128', 's30-b32-l512', 's30-b128-l512', 's30-b32-l1024','s30-b128-l1024']
#target_list = ['ts', 'GPUBusy', 'VALUBusy', 'MemUnitBusy', 'VALUUtilization', 'ratio1', 'ratio2']
target_list = ['ts']
cumulative_index = 0
for model_index in range(len(df_dict_updated)):#range(len(workloads)): #For each model
    df_original2 = df_dict_updated[model_index]
    for target in target_list:
        df_original3 = df_original2[feature_list[model_index]]
        ##Make sure to fill up all Nan values with 0
        df_original3 = df_original3.fillna(0)

        #Add a target at a time to original3
        df_original3[target] = df_original2[target]

        #
        outfilename = output_prefix[ind] + '_' + target +'_'+filelist[model_index]+'.csv'
        #print('Generating ' + outfilename + '...')
        generate_dash_data_format(df_original3, feature_list, target, region_list, outfilename, "KernelName")
        ## TODO: PASS the following function feature_list[ind] and region_list[ind]
        #generate_dash_data_format(df_original3, feature_list[model_index], target, region_list[model_index], outfilename, "KernelName")
        print('    - ', filelist[model_index])
        #generate_config_block_for_this_input(filelist[model_index], outfilename, target)
print("Done")

In [ ]:
########## AMD Q2 -- feature compression and feature correaltion analysis --- 
## input: df_dict_updated --> each model has all input combined into it.
colors=['red', 'blue', 'green', 'orange']

for model_index in range(len(df_dict_updated)):#range(len(workloads)): #For each model
    df_original2 = df_dict_updated[model_index]
    plot_df(df_original2, 'ts', colors[model_index])


In [ ]:
def plot_df(input_df, target, c):
    fig, ax = plt.subplots()
    input_df[target].plot(ax=ax, kind='bar', color = c)

In [ ]:
# Returns info sieve output with nh components    
def infoSieveX(X,nh):
        identifier= 'Using Information Sieve %d Components'%nh
        s = linearsieve.Sieve(n_hidden=nh, verbose=0).fit(X + 0.05*np.random.randn(X.shape[0],X.shape[1]))
        X_lat = s.transform(X) 
        return X_lat

In [ ]:
df2 = df_dict_updated[0]
col_std_list = [ col for col in df2.columns if '_std' in col]
for c in col_std_list:
    print(c)

In [ ]:
df = pd.read_csv('in.txt', sep=" ")
file1 = open("event_names_amd.txt","w")

for k in df:
    #print (k)
    file1.write(k+"\n")
file1.close()
        

In [ ]:
############## Special processing block for Q2: AMD report
##Input: res_imp_rnn.csv file which contains resource importance for all the kernels. 


In [34]:
def read_fom_lstm_output(filename):

    ###### Read and parse FOM for LSTM / RNN benchmark's output
    total = 0.0
    count = 0
    with open(filename, "r") as fom_file:
        read_next_line = 0
        for line in fom_file:
    #         print(line)
            if read_next_line == 1: #If I am supposed to read this line 
                total += np.float((line.split())[4])
                read_next_line = 0
                count += 1
            elif "Forward + Backward" in line:
                read_next_line = 1
    return 1.0/(total/count)
        
        

In [11]:
# read_fom_lstm_output('/Users/tanzima/Research/AMD/DATA/mi-50/DATA-aac-july21/results')
import os
model_names = ['basic_lstm', 'rnn', 'lstm']
for filename in os.listdir('/Users/tanzima/Research/AMD/DATA/mi-100/prof/adam/basic_lstm'):
    print ("'"+filename+"',")
    

'prof-mi100-basic_lstm-adam-b4-l512-s25.csv',
'prof-mi100-basic_lstm-adam-b4-l64-s25.csv',
'prof-mi100-basic_lstm-adam-b8-l256-s25.csv',
'prof-mi100-basic_lstm-adam-b8-l64-s25.csv',
'prof-mi100-basic_lstm-adam-b8-l1024-s25.csv',
'prof-mi100-basic_lstm-adam-b8-l512-s25.csv',
'prof-mi100-basic_lstm-adam-b8-l32-s25.csv',
'prof-mi100-basic_lstm-adam-b4-l256-s25.csv',
'prof-mi100-basic_lstm-adam-b4-l32-s25.csv',
'prof-mi100-basic_lstm-adam-b4-l1024-s25.csv',
